# Code for interacting with API of openFDA

## Plan
In this project, we are looking for device adverse events. We are particularly interested in:
    - Cause of failure
    - Date of failure (ie age of device)
We want to see which is the most common cause for given device to fail and at which stage of use this occurs.
Relevant fields could be (for full reference see https://open.fda.gov/device/event/reference/):
### Event
device_date_of_manufacturer
date_of_event, date_report, date_received
previous_use_code, remedial_action
single_use_flag
### Source
reprocessed_and_reused_flag
### Device
device.generic_name
device.expiration_date_of_device, device.device_age_text
device.implant_flag, device.date_removed_flag
device.manufacturer_d_name, device.manufacturer_d_state, device.manufacturer_d_country
### Patient
patient.sequence_number_outcome, patient.sequence_number_treatment
### Report Text
mdr_text.text, mdr_text.text_type_code
### Reporter Dependent Fields
#### By user facility / importer
report_date
event_location
manufacturer_name, manufacturer_country
manufacturer_g1_name, manufacturer_g1_state
### OpenFDA fields
device_class
### Further interesting fields:
Source: reporter_occupation_code
Device: device.device_operator

In [1]:
import numpy as np
import pandas as pd
import json
import requests

fileName = 'device-event-2016Q1-JSON~\device-event-0001-of-0002.json'
baseurl = 'https://api.fda.gov/device/event.json'

In [97]:
# Example of quering, for complete guide go to: https://open.fda.gov/api/
q1 = dataurl + '?search=previous_use_code:"R"&limit=1'
dq1 = requests.get(q1)
# dq1.json()['results']
data = json.loads(dq1.text)
results = data['results']

In [98]:
# Fields of Interest
fois_device = ['generic_name', 'expiration_date_of_device', 'device_age_text', 'implant_flag', 'date_removed_flag', \
               'manufacturer_d_name', 'manufacturer_d_state', 'manufacturer_d_country']
fois_patient = ['sequence_number_outcome', 'sequence_number_treatment']
fois_mdrText = ['text', 'text_type_code']
fois_openfda = ['device_name', 'device_class', 'medical_specialty_description']
# Currently works only for single item query! - extend to encopass more than limit=1
device = data['results'][0]['device'][0]
patient = data['results'][0]['patient'][0]
mdrText = data['results'][0]['mdr_text'][0] # there may be more items in the list! 
openfda = data['results'][0]['device'][0]['openfda']

In [101]:
df_openfda = pd.DataFrame(openfda, index = range(len(results)),columns = fois_openfda)
df_device = pd.DataFrame(device, index = range(len(results)),columns = fois_device)
df_patient = pd.DataFrame(patient, index = range(len(results)),columns = fois_patient)
df_mdrText = pd.DataFrame(mdrText, index = range(len(results)),columns = fois_mdrText)
df = pd.concat([df_device, df_patient, df_mdrText, df_openfda], axis = 1)
df